<a href="https://colab.research.google.com/github/ccastano1997/version3.1/blob/main/56k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# 1. Load the file
df = pd.read_csv("nba_odds_full.csv")

# 2. Define the updates for Dec 4th (Scores found in the previous images/PDF)
# Format in CSV is "HomeScore-AwayScore" based on existing data like "129-126"
dec4_updates = {
    "Washington Wizards": "101-146",
    "Philadelphia 76ers": "99-98",
    "Toronto Raptors": "120-123",
    "Brooklyn Nets": "110-123",
    "New Orleans Pelicans": "116-125"
}

# 3. Update the rows
# We iterate through the updates and apply them to the rows matching Date="04 Dec 2025" and Home Team
for home_team, score in dec4_updates.items():
    mask = (df['Date'] == '04 Dec 2025') & (df['Home Team'] == home_team)
    df.loc[mask, 'Score'] = score

# 4. Append Dec 5th Games (Since they were missing in the .tail() output)
# Data from page8.pdf
dec5_games = [
    ["05 Dec 2025", "19:00", "Boston Celtics", "Los Angeles Lakers", "", "-250", "+200", 6],
    ["05 Dec 2025", "19:00", "Orlando Magic", "Miami Heat", "", "-208", "+171", 6],
    ["05 Dec 2025", "19:30", "Atlanta Hawks", "Denver Nuggets", "", "+199", "-244", 6],
    ["05 Dec 2025", "19:30", "Cleveland Cavaliers", "San Antonio Spurs", "", "-196", "+164", 6],
    ["05 Dec 2025", "19:30", "Detroit Pistons", "Portland Trail Blazers", "", "-303", "+240", 6],
    ["05 Dec 2025", "19:30", "New York Knicks", "Utah Jazz", "", "-1250", "+705", 6],
    ["05 Dec 2025", "19:30", "Charlotte Hornets", "Toronto Raptors", "", "-312", "+244", 5],
    ["05 Dec 2025", "20:00", "Chicago Bulls", "Indiana Pacers", "", "-196", "+163", 6],
    ["05 Dec 2025", "20:00", "Houston Rockets", "Phoenix Suns", "", "-556", "+399", 6],
    ["05 Dec 2025", "20:00", "Memphis Grizzlies", "Los Angeles Clippers", "", "+106", "-127", 6],
    ["05 Dec 2025", "20:00", "Milwaukee Bucks", "Philadelphia 76ers", "", "+104", "-123", 6],
    ["05 Dec 2025", "21:30", "Oklahoma City Thunder", "Dallas Mavericks", "", "-1111", "+684", 6]
]

# Check if Dec 5 is already there to avoid duplicates
if not df['Date'].str.contains('05 Dec 2025').any():
    new_df = pd.DataFrame(dec5_games, columns=df.columns)
    df = pd.concat([df, new_df], ignore_index=True)

# 5. Save the fixed file
df.to_csv("nba_odds_full.csv", index=False)

print("Updated Dec 4th scores and added Dec 5th games.")
print(df.tail(15)) # Verify the end of the file

Updated Dec 4th scores and added Dec 5th games.
            Date   Time              Home Team               Away Team  \
290  04 Dec 2025  19:30        Toronto Raptors      Los Angeles Lakers   
291  04 Dec 2025  19:30          Brooklyn Nets               Utah Jazz   
292  04 Dec 2025  20:00   New Orleans Pelicans  Minnesota Timberwolves   
293  05 Dec 2025  19:00         Boston Celtics      Los Angeles Lakers   
294  05 Dec 2025  19:00          Orlando Magic              Miami Heat   
295  05 Dec 2025  19:30          Atlanta Hawks          Denver Nuggets   
296  05 Dec 2025  19:30    Cleveland Cavaliers       San Antonio Spurs   
297  05 Dec 2025  19:30        Detroit Pistons  Portland Trail Blazers   
298  05 Dec 2025  19:30        New York Knicks               Utah Jazz   
299  05 Dec 2025  19:30      Charlotte Hornets         Toronto Raptors   
300  05 Dec 2025  20:00          Chicago Bulls          Indiana Pacers   
301  05 Dec 2025  20:00        Houston Rockets            Phoeni

In [2]:
!pip install nba_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.0/319.0 kB 8.6 MB/s eta 0:00:00


In [4]:
#==========================================
# 🏀 NBA PREDICTION MODEL V3.0 (Advanced)
# ==========================================
# Features: LSTM Neural Network, Four Factors Analytics, Live Odds, Injury Audit
# ==========================================

import pandas as pd
import numpy as np
import time
from nba_api.stats.endpoints import leaguegamefinder, leaguedashplayerstats
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras import regularizers

# --- CONFIGURATION ---
SEASON = '2025-26'
SEQ_LENGTH = 5  # Games to look back
ODDS_FILE = 'nba_odds_full.csv'

# ==========================================
# 1. LOAD AND PROCESS ODDS DATA
# ==========================================
print(f"--- STEP 1: Loading Betting Data ({ODDS_FILE}) ---")
try:
    odds_df = pd.read_csv(ODDS_FILE)
    odds_df['Date'] = pd.to_datetime(odds_df['Date'], format='%d %b %Y')
except FileNotFoundError:
    print(f"❌ Error: '{ODDS_FILE}' not found. Please upload it.")
    raise

# Convert American Odds to Implied Probability
def american_to_prob(odd_str):
    try:
        if pd.isna(odd_str) or odd_str == 'v': return 0.5
        odd = float(odd_str)
        if odd > 0: return 100 / (odd + 100)
        else: return abs(odd) / (abs(odd) + 100)
    except: return 0.5

odds_df['Home_Prob'] = odds_df['Home Odds'].apply(american_to_prob)
odds_df['Away_Prob'] = odds_df['Away Odds'].apply(american_to_prob)

# Split Training (Past Results) vs Prediction (Tonight)
train_odds = odds_df[odds_df['Score'] != 'v'].copy()
# Handles 'v', empty strings, or NaN (Empty cells)
predict_odds = odds_df[odds_df['Score'].isna() | (odds_df['Score'] == 'v') | (odds_df['Score'] == '')].copy()
print(f"✅ Loaded {len(train_odds)} past games for training.")
print(f"✅ Found {len(predict_odds)} games to predict tonight.")

--- STEP 1: Loading Betting Data (nba_odds_full.csv) ---
✅ Loaded 305 past games for training.
✅ Found 12 games to predict tonight.


In [8]:
import pandas as pd
import requests
import io

def get_live_injury_report():
    print("🏥 Scrapping Live Injury Report...")
    url = "https://www.espn.com/nba/injuries"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers)
        dfs = pd.read_html(io.StringIO(response.text))

        injured_stars = {}

        # ESPN tables are grouped by Team. We iterate through them.
        for df in dfs:
            # Clean the table
            if df.empty or len(df.columns) < 2: continue
            df.columns = ['NAME', 'STATUS', 'REASON'] # Standardize

            # Filter for "OUT" or "DOUBTFUL"
            # You can adjust this list to include "Day-To-Day" if you want to be cautious
            out_players = df[df['STATUS'].str.contains('Out|Doubtful|Questionable', case=False, na=False)]

            for _, row in out_players.iterrows():
                player_name = row['NAME']
                status = row['STATUS']

                # OPTIONAL: Filter for only "Stars" (Players > 20 PPG or high usage)
                # For now, we capture everyone, but you can add a filter here.
                # impact_score = get_player_war(player_name) # Assuming you have a WAR function
                # if impact_score > 5.0:

                # We need to map the player to their Team.
                # (Simple way: store them in a list and fuzzy match later)
                injured_stars[player_name] = status

        print(f"✅ Found {len(injured_stars)} active injuries.")
        return injured_stars

    except Exception as e:
        print(f"❌ Injury Scrape Failed: {e}")
        return {}

In [6]:
from nba_api.stats.endpoints import teamgamelog, playergamelog
import time

def calculate_team_resilience(team_name, player_name, season='2025-26'):
    # 1. Get Team ID and Player ID (You need a helper for this mapping)
    team_id = get_team_id(team_name)
    player_id = get_player_id(player_name)

    if not team_id or not player_id: return None

    # 2. Fetch Logs
    # (Add time.sleep to be polite to API)
    time.sleep(0.5)
    t_log = teamgamelog.TeamGameLog(team_id=team_id, season=season).get_data_frames()[0]
    time.sleep(0.5)
    p_log = playergamelog.PlayerGameLog(player_id=player_id, season=season).get_data_frames()[0]

    # 3. Find Games WITHOUT the Player
    games_with_player = set(p_log['Game_ID'])
    games_without = t_log[~t_log['Game_ID'].isin(games_with_player)]

    if games_without.empty:
        return {"status": "Unknown", "record": "0-0", "win_pct": 0.0}

    # 4. Calculate Win %
    wins = len(games_without[games_without['WL'] == 'W'])
    losses = len(games_without[games_without['WL'] == 'L'])
    total = wins + losses
    win_pct = wins / total

    # 5. Classify
    status = "Neutral"
    if win_pct > 0.60: status = "Resilient"  # e.g., Lakers 3-1
    elif win_pct < 0.30: status = "Dependent" # e.g., Hawks 0-4

    print(f"   📊 Analysis: {team_name} are {wins}-{losses} without {player_name}")
    return {"status": status, "record": f"{wins}-{losses}", "win_pct": win_pct}

In [7]:
def build_live_resilience_map(games_tonight):
    print("\n--- 🤖 BUILDING LIVE RESILIENCE MAP ---")

    # 1. Get Injuries
    injuries = get_live_injury_report() # From Step 1

    resilience_map = {}

    # 2. Check only teams playing TONIGHT
    for game in games_tonight:
        for team in [game['Home Team'], game['Away Team']]:

            # Find star players for this team who are injured
            # (You need a roster list to match "Luka Doncic" to "Lakers")
            team_stars = get_team_stars(team) # e.g., ['Luka Doncic', 'LeBron James']

            for star in team_stars:
                if star in injuries:
                    print(f"⚠️ Checking Impact: {star} is {injuries[star]} for {team}...")

                    # 3. Run the "Ewing Calculation"
                    data = calculate_team_resilience(team, star)

                    if data:
                        resilience_map[team] = {
                            "star_out": True,
                            "player": star,
                            "status": data['status'],
                            "resilience_score": data['win_pct']
                        }

    print("✅ Resilience Map Built.")
    return resilience_map

In [9]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguedashplayerstats
import pandas as pd
import time

def get_dynamic_star_rosters(teams_playing_list):
    """
    Returns a dict of stars for teams playing tonight.
    Structure: {'Los Angeles Lakers': ['Luka Doncic', 'LeBron James'], ...}
    """
    print("\n📊 CALCULATING PLAYER IMPACT (WAR)...")

    # 1. Fetch Season Stats
    time.sleep(1) # Be polite to API
    p_stats = leaguedashplayerstats.LeagueDashPlayerStats(season=SEASON).get_data_frames()[0]

    # 2. Calculate Custom Impact Score (Your Formula)
    p_stats['IMPACT'] = (
        p_stats['PTS'] * 1.0 +
        p_stats['REB'] * 1.2 +
        p_stats['AST'] * 1.5 +
        p_stats['STL'] * 2.0 +
        p_stats['BLK'] * 2.0 -
        p_stats['TOV'] * 1.5
    ) / p_stats['GP']

    # Normalize to 0-100 Scale
    max_score = p_stats['IMPACT'].max()
    p_stats['WAR'] = (p_stats['IMPACT'] / max_score) * 100

    # 3. Create Mapping: Full Name -> Abbreviation (e.g. "Los Angeles Lakers" -> "LAL")
    nba_teams = teams.get_teams()
    team_map = {team['full_name']: team['abbreviation'] for team in nba_teams}

    # Fix common CSV name mismatches
    team_map['Los Angeles Clippers'] = 'LAC'
    team_map['LA Clippers'] = 'LAC'

    star_rosters = {}

    # 4. Filter for tonight's teams
    for team_name in teams_playing_list:
        abbr = team_map.get(team_name)
        if not abbr:
            print(f"⚠️ Warning: No abbreviation found for {team_name}")
            continue

        # Get Top 3 Players by WAR for this team
        roster = p_stats[p_stats['TEAM_ABBREVIATION'] == abbr].sort_values('WAR', ascending=False).head(3)

        # Only count them as "Stars" if WAR > 65 (Filter out leaders of bad teams)
        stars = roster[roster['WAR'] > 65]['PLAYER_NAME'].tolist()
        star_rosters[team_name] = stars

        if stars:
            print(f"   🌟 {team_name} Stars: {', '.join(stars)}")

    return star_rosters

In [ ]:
# ==========================================
# 5. PREDICT TONIGHT'S GAMES (FINAL V3.1)
# ==========================================
print("\n=======================================================")
print("🤖  INITIATING MODEL V3.1: DYNAMIC IMPACT ANALYSIS")
print("=======================================================")

# -------------------------------------------------------
# A. DYNAMIC STAR IDENTIFICATION
# -------------------------------------------------------
# 1. Get list of all teams playing tonight
teams_tonight = pd.concat([predict_odds['Home Team'], predict_odds['Away Team']]).unique()

# 2. Find out who the "Stars" are statistically right now
STAR_ROSTERS = get_dynamic_star_rosters(teams_tonight)

# -------------------------------------------------------
# B. BUILD LIVE RESILIENCE MAP
# -------------------------------------------------------
# Now we update our builder to use STAR_ROSTERS instead of a hardcoded list
def build_dynamic_resilience_map(games_df, star_rosters):
    print("\n🏥 CHECKING INJURY STATUS OF IDENTIFIED STARS...")
    injuries = get_live_injury_report() # Your scraper function
    resilience_map = {}

    for _, row in games_df.iterrows():
        for team in [row['Home Team'], row['Away Team']]:
            stars = star_rosters.get(team, [])

            for star in stars:
                # Check if this specific star is OUT
                if star in injuries:
                    print(f"⚠️ MATCH: {star} (WAR Leader) is {injuries[star]} for {team}")

                    # Run the "Ewing Theory" History Check
                    # (Uses your calculate_team_resilience function)
                    data = calculate_team_resilience(team, star)

                    if data:
                        resilience_map[team] = {
                            "star_out": True,
                            "player": star,
                            "status": data['status'],
                            "resilience_score": data['win_pct']
                        }
    return resilience_map

# Build the map using the stars we just found
LIVE_RESILIENCE_MAP = build_dynamic_resilience_map(predict_odds, STAR_ROSTERS)

# -------------------------------------------------------
# C. RUN PREDICTIONS
# -------------------------------------------------------
print("\n--- 🏀 GENERATING PREDICTIONS ---")
print(f"{'MATCHUP':<40} | {'SPREAD':<6} | {'SIGNAL':<25} | {'LOGIC'}")
print("-" * 110)

for _, row in predict_odds.iterrows():
    home, away = row['Home Team'], row['Away Team']

    # 1. Prepare Stats Input (LSTM)
    home_stats = games[games['TEAM_NAME'] == home].sort_values('GAME_DATE').tail(SEQ_LENGTH)
    input_seq = home_stats[features].values
    input_seq = scaler.transform(input_seq).reshape(1, SEQ_LENGTH, len(features))
    win_prob = model.predict(input_seq, verbose=0)[0][0]

    # 2. Get Implied Spread
    implied_spread = get_implied_spread(row['Home Odds'])

    # 3. Apply Resilience Logic
    signal = "-- No Bet --"
    logic_note = f"Model Prob: {win_prob:.2f}"

    # Check Home
    if home in LIVE_RESILIENCE_MAP:
        data = LIVE_RESILIENCE_MAP[home]
        if data['status'] == 'Resilient':
            signal = f"💰 BET {home} (Value)"
            logic_note = f"💎 3-1 w/o {data['player']}"
        elif data['status'] == 'Dependent':
            signal = f"📉 FADE {home}"
            logic_note = f"❌ Needs {data['player']}"

    # Check Away
    elif away in LIVE_RESILIENCE_MAP:
        data = LIVE_RESILIENCE_MAP[away]
        if data['status'] == 'Resilient':
            signal = f"💰 BET {away} (Value)"
            logic_note = f"💎 {away} Resilient w/o {data['player']}"
        elif data['status'] == 'Dependent':
            signal = f"📉 FADE {away}"
            logic_note = f"❌ Needs {data['player']}"

    # 4. Standard Model Logic
    if signal == "-- No Bet --":
        if win_prob > 0.60: signal = f"🔥 BET {home}"
        elif win_prob < 0.40: signal = f"🔥 BET {away}"
        elif abs(implied_spread) > 14: signal = "PASS"

    print(f"{away} @ {home:<15} | {implied_spread:<6} | {signal:<25} | {logic_note}")

In [10]:
# ==========================================
# 2. FETCH NBA STATS & FEATURE ENGINEERING
# ==========================================
print(f"\n--- STEP 2: Fetching {SEASON} Game Stats ---")
gamefinder = leaguegamefinder.LeagueGameFinder(
    season_nullable=SEASON,
    league_id_nullable='00',
    season_type_nullable='Regular Season'
)
all_games = gamefinder.get_data_frames()[0]
all_games['GAME_DATE'] = pd.to_datetime(all_games['GAME_DATE'])
all_games = all_games.sort_values('GAME_DATE')
all_games = all_games[all_games['WL'].notna()] # Remove games not played yet

# --- CALCULATE "FOUR FACTORS" (Advanced Stats) ---
# 1. Effective Field Goal % (Adjusts for 3-pointers)
all_games['EFG_PCT'] = (all_games['FGM'] + 0.5 * all_games['FG3M']) / all_games['FGA']

# 2. Turnover Rate (Turnovers per 100 possessions)
# Estimate Possessions: FGA + 0.44*FTA - OREB + TOV
all_games['POSS_EST'] = all_games['FGA'] + 0.44 * all_games['FTA'] - all_games['OREB'] + all_games['TOV']
all_games['TOV_RATE'] = 100 * (all_games['TOV'] / all_games['POSS_EST'])

# 3. Offensive Rebounding (Raw proxy since we lack opp stats in this view)
all_games['OREB_RATE'] = all_games['OREB'] / all_games['POSS_EST'] # Proxy

# 4. Free Throw Rate (Ability to draw fouls)
all_games['FTA_RATE'] = all_games['FTA'] / all_games['FGA']

# 5. Pace (Speed of play)
all_games['PACE'] = 48 * (all_games['POSS_EST'] / (all_games['MIN'].astype(float) / 5))

# Target Variable
all_games['TARGET'] = all_games['WL'].apply(lambda x: 1 if x == 'W' else 0)

print("✅ Advanced Stats Calculated (Four Factors).")


--- STEP 2: Fetching 2025-26 Game Stats ---
✅ Advanced Stats Calculated (Four Factors).


In [11]:
#==========================================
# 3. MERGE API STATS WITH ODDS
# ==========================================
name_map = {
    'LA Clippers': 'Los Angeles Clippers',
    'L.A. Clippers': 'Los Angeles Clippers',
    'L.A. Lakers': 'Los Angeles Lakers'
}
all_games['TEAM_NAME'] = all_games['TEAM_NAME'].replace(name_map)

# Create Odds Lookup Table
odds_lookup = pd.concat([
    train_odds[['Date', 'Home Team', 'Home_Prob']].rename(columns={'Date':'GAME_DATE', 'Home Team':'TEAM_NAME', 'Home_Prob':'ODDS_PROB'}),
    train_odds[['Date', 'Away Team', 'Away_Prob']].rename(columns={'Date':'GAME_DATE', 'Away Team':'TEAM_NAME', 'Away_Prob':'ODDS_PROB'})
])

# Merge
merged_data = pd.merge(all_games, odds_lookup, on=['GAME_DATE', 'TEAM_NAME'], how='left')
merged_data['ODDS_PROB'] = merged_data['ODDS_PROB'].fillna(0.5)

# Define the V2 Feature Set
features = [
    'ODDS_PROB',    # Market Sentiment (Crucial)
    'EFG_PCT',      # Shooting Efficiency
    'TOV_RATE',     # Ball Security
    'OREB_RATE',    # Rebounding
    'FTA_RATE',     # Aggression
    'PACE',         # Tempo
    'PLUS_MINUS'    # Overall Dominance
]

In [12]:
# ==========================================
# 4. TRAIN LSTM MODEL (V3 ARCHITECTURE)
# ==========================================
print("\n--- STEP 4: Training LSTM Neural Network ---")

# 1. Scale Data (0 to 1) for the Neural Network
# We use the 'features' list you defined in Step 3
scaler = MinMaxScaler()
merged_data[features] = scaler.fit_transform(merged_data[features])

# 2. Create Sequences (The "Memory" of the Model)
# We feed the model the LAST 5 games to predict the NEXT game result
X, y = [], []

# Iterate through each unique team in the dataset
for team_id in merged_data['TEAM_ID'].unique():
    team_df = merged_data[merged_data['TEAM_ID'] == team_id].sort_values('GAME_DATE')

    # Skip teams with insufficient data
    if len(team_df) < SEQ_LENGTH + 1: continue

    vals = team_df[features].values
    # Convert 'W'/'L' to 1/0
    targets = team_df['WL'].apply(lambda x: 1 if x == 'W' else 0).values

    # Create sequences: Use 5 games (SEQ_LENGTH) to predict the 6th
    for i in range(len(team_df) - SEQ_LENGTH):
        X.append(vals[i:i+SEQ_LENGTH])      # Sequence of 5 games
        y.append(targets[i+SEQ_LENGTH])     # Result of the 6th game

X, y = np.array(X), np.array(y)

# 3. Build LSTM Architecture (V3)
# This is a Bidirectional LSTM which is more powerful than standard LSTM
model = Sequential([
    # Layer 1: Bidirectional LSTM
    Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)),
                  input_shape=(SEQ_LENGTH, len(features))),
    Dropout(0.3), # Prevents overfitting

    # Layer 2: Second LSTM Layer
    Bidirectional(LSTM(32)),
    Dropout(0.3),

    # Layer 3: Output
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid') # Output: Probability (0 to 1)
])

# 4. Train the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Verbose=0 keeps the output clean
model.fit(X, y, epochs=30, batch_size=16, verbose=0)

print(f"✅ Model Trained on {len(X)} historical game sequences.")
print(f"✅ Input Shape: {X.shape} (Samples, Time Steps, Features)")


--- STEP 4: Training LSTM Neural Network ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


✅ Model Trained on 513 historical game sequences.
✅ Input Shape: (513, 5, 7) (Samples, Time Steps, Features)


In [13]:
# ==========================================
# 5. PREDICT TONIGHT'S GAMES (FINAL V3.1)
# ==========================================
print("\n=======================================================")
print("🤖  INITIATING MODEL V3.1: DYNAMIC IMPACT ANALYSIS")
print("=======================================================")

# -------------------------------------------------------
# A. DYNAMIC STAR IDENTIFICATION
# -------------------------------------------------------
# 1. Get list of all teams playing tonight
teams_tonight = pd.concat([predict_odds['Home Team'], predict_odds['Away Team']]).unique()

# 2. Find out who the "Stars" are statistically right now
STAR_ROSTERS = get_dynamic_star_rosters(teams_tonight)

# -------------------------------------------------------
# B. BUILD LIVE RESILIENCE MAP
# -------------------------------------------------------
# Now we update our builder to use STAR_ROSTERS instead of a hardcoded list
def build_dynamic_resilience_map(games_df, star_rosters):
    print("\n🏥 CHECKING INJURY STATUS OF IDENTIFIED STARS...")
    injuries = get_live_injury_report() # Your scraper function
    resilience_map = {}

    for _, row in games_df.iterrows():
        for team in [row['Home Team'], row['Away Team']]:
            stars = star_rosters.get(team, [])

            for star in stars:
                # Check if this specific star is OUT
                if star in injuries:
                    print(f"⚠️ MATCH: {star} (WAR Leader) is {injuries[star]} for {team}")

                    # Run the "Ewing Theory" History Check
                    # (Uses your calculate_team_resilience function)
                    data = calculate_team_resilience(team, star)

                    if data:
                        resilience_map[team] = {
                            "star_out": True,
                            "player": star,
                            "status": data['status'],
                            "resilience_score": data['win_pct']
                        }
    return resilience_map

# Build the map using the stars we just found
LIVE_RESILIENCE_MAP = build_dynamic_resilience_map(predict_odds, STAR_ROSTERS)

# -------------------------------------------------------
# C. RUN PREDICTIONS
# -------------------------------------------------------
print("\n--- 🏀 GENERATING PREDICTIONS ---")
print(f"{'MATCHUP':<40} | {'SPREAD':<6} | {'SIGNAL':<25} | {'LOGIC'}")
print("-" * 110)

for _, row in predict_odds.iterrows():
    home, away = row['Home Team'], row['Away Team']

    # 1. Prepare Stats Input (LSTM)
    home_stats = games[games['TEAM_NAME'] == home].sort_values('GAME_DATE').tail(SEQ_LENGTH)
    input_seq = home_stats[features].values
    input_seq = scaler.transform(input_seq).reshape(1, SEQ_LENGTH, len(features))
    win_prob = model.predict(input_seq, verbose=0)[0][0]

    # 2. Get Implied Spread
    implied_spread = get_implied_spread(row['Home Odds'])

    # 3. Apply Resilience Logic
    signal = "-- No Bet --"
    logic_note = f"Model Prob: {win_prob:.2f}"

    # Check Home
    if home in LIVE_RESILIENCE_MAP:
        data = LIVE_RESILIENCE_MAP[home]
        if data['status'] == 'Resilient':
            signal = f"💰 BET {home} (Value)"
            logic_note = f"💎 3-1 w/o {data['player']}"
        elif data['status'] == 'Dependent':
            signal = f"📉 FADE {home}"
            logic_note = f"❌ Needs {data['player']}"

    # Check Away
    elif away in LIVE_RESILIENCE_MAP:
        data = LIVE_RESILIENCE_MAP[away]
        if data['status'] == 'Resilient':
            signal = f"💰 BET {away} (Value)"
            logic_note = f"💎 {away} Resilient w/o {data['player']}"
        elif data['status'] == 'Dependent':
            signal = f"📉 FADE {away}"
            logic_note = f"❌ Needs {data['player']}"

    # 4. Standard Model Logic
    if signal == "-- No Bet --":
        if win_prob > 0.60: signal = f"🔥 BET {home}"
        elif win_prob < 0.40: signal = f"🔥 BET {away}"
        elif abs(implied_spread) > 14: signal = "PASS"

    print(f"{away} @ {home:<15} | {implied_spread:<6} | {signal:<25} | {logic_note}")


🤖  INITIATING MODEL V3.1: DYNAMIC IMPACT ANALYSIS

📊 CALCULATING PLAYER IMPACT (WAR)...
   🌟 Boston Celtics Stars: Jaylen Brown
   🌟 Atlanta Hawks Stars: Jalen Johnson
   🌟 Cleveland Cavaliers Stars: Donovan Mitchell
   🌟 Detroit Pistons Stars: Cade Cunningham
   🌟 New York Knicks Stars: Karl-Anthony Towns, Jalen Brunson
   🌟 Chicago Bulls Stars: Josh Giddey
   🌟 Houston Rockets Stars: Alperen Sengun
   🌟 Milwaukee Bucks Stars: Giannis Antetokounmpo
   🌟 Oklahoma City Thunder Stars: Shai Gilgeous-Alexander
   🌟 Los Angeles Lakers Stars: Luka Dončić, Austin Reaves
   🌟 Denver Nuggets Stars: Nikola Jokić, Jamal Murray
   🌟 San Antonio Spurs Stars: Victor Wembanyama
   🌟 Portland Trail Blazers Stars: Deni Avdija
   🌟 Utah Jazz Stars: Lauri Markkanen
   🌟 Toronto Raptors Stars: Scottie Barnes
   🌟 Indiana Pacers Stars: Pascal Siakam
   🌟 Los Angeles Clippers Stars: James Harden
   🌟 Philadelphia 76ers Stars: Tyrese Maxey
   🌟 Dallas Mavericks Stars: Anthony Davis

🏥 CHECKING INJURY STATUS

NameError: name 'games' is not defined